In [1]:
# import gdown ; 
import pandas as pd ; 
import numpy as np;
import math
import os
# train_log_url = "https://drive.google.com/file/d/1VebJgr8LlRWeCsAnYA2OK6VzdtsB9L83/view?usp=drive_link"
# gdown.download(url=train_log_url, quiet=False, fuzzy=True)

# train_score_url = "https://drive.google.com/file/d/1VQ05dODI0aKyvu2RB0-WU2hzk4_bqJyw/view?usp=sharing"
# gdown.download(url=train_score_url, quiet=False, fuzzy=True)

In [2]:
train_logs = pd.read_csv("train_logs.csv")
train_score = pd.read_csv("train_scores.csv")
## number of distinct downevent
print(train_logs["down_event"].nunique()) ## 131
print(train_logs["activity"].nunique()) ## 50
train_logs.head(10)

131
50


,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
5,001519c8,6,107296,107400,104,Input,q,q,q,3,1
6,001519c8,7,107469,107596,127,Input,q,q,q,4,1
7,001519c8,8,107659,107766,107,Input,q,q,q,5,1
8,001519c8,9,107743,107852,109,Input,q,q,q,6,1
9,001519c8,10,107840,107978,138,Input,Space,Space,,7,1


In [3]:
##function to apply on each row for train_logs
def pause_time_to_string(value):
    ## If the pause is bigger than 1000(1sec), value/1000 that many pause to column
    if math.isnan(value)or value <= 1000 :
        return ""
    else:
        sec = round(value/ 1000)
        return " ".join(['pause' for i in range(sec)])


In [4]:
def genHMMInput(df):
    df = df.copy()
    ##create a new coulmn that contains pause time
    df["next_down_time"] = df["down_time"].shift(-1)
    df["pause_time"] = df["next_down_time"] - df["down_time"]
    ####create new column pause_str.
    df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
    df["hmm_input"] = df["down_event"] + " " + df["pause_str"]
    hmm_input = " ".join(df["hmm_input"].tolist())
    return hmm_input

In [5]:
splitted_df = []
##generate inner join dataframe
log_join_score = pd.merge(train_logs, train_score, on="id", how='inner')
for s in train_score["score"].unique():
    splitted_df.append((s, log_join_score.loc[log_join_score["score"] == s]))


In [8]:
if not os.path.exists("hmm"):
    os.makedirs("hmm")
for s, df in splitted_df:
    hmm_input = genHMMInput(df)
    s = str(s)
    print(s)
    if not os.path.exists("hmm/"+s):
        os.makedirs("hmm/"+s)
    with open("hmm/"+s+"/" +s +".txt", 'w') as file:
        file.write(hmm_input)
    
    

3.5
6.0
2.0
4.0
4.5
2.5
5.0
3.0
1.5
5.5
1.0
0.5


['Leftclick',
 'Leftclick',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 'pause',
 '

102

In [43]:
len(hmm_input)

10704286

In [ ]:
column_name = "pause_time"
# plt.scatter(train_logs.index, train_logs[column_name])

# # Add labels and title
# plt.xlabel('Index')
# plt.ylabel(column_name)
# plt.title(f'Scatter Plot of {column_name} vs Index')

# # Show the plot
# # plt.show()

percentiles = [25,50,75,80,85] + list(range(90,101))
# percentiles = list(range(90,101))

# Use the percentile function to calculate the specified percentiles
percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
for p, value in zip(percentiles, percentile_values):
    print(f'{p}th percentile: {value}')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming you already have a DataFrame named 'df'
# Replace 'column_name' with the actual column you want to plot
column_name = 'action_time'
# train_logs["action_time"] = train_logs["action_time"].astype(int)
mask = (train_logs["action_time"] <=  10000)
df = train_logs[mask]
# Create a scatter plot
plt.scatter(df.index, df[column_name])

# Add labels and title
plt.xlabel('Index')
plt.ylabel(column_name)
plt.title(f'Scatter Plot of {column_name} vs Index')

# Show the plot
plt.show()

percentiles = list(range(90,101))

# Use the percentile function to calculate the specified percentiles
percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
for p, value in zip(percentiles, percentile_values):
    print(f'{p}th percentile: {value}')

In [ ]:
# Assuming you already have a DataFrame named 'df'
# Replace 'column_name' with the actual column you want to plot
column_name = ''
# train_logs["action_time"] = train_logs["action_time"].astype(int)
mask = (train_logs["action_time"] <=  10000)
df = train_logs[mask]
# Create a scatter plot
plt.scatter(df.index, df[column_name])

# Add labels and title
plt.xlabel('Index')
plt.ylabel(column_name)
plt.title(f'Scatter Plot of {column_name} vs Index')

# Show the plot
plt.show()

percentiles = list(range(90,101))

# Use the percentile function to calculate the specified percentiles
percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
for p, value in zip(percentiles, percentile_values):
    print(f'{p}th percentile: {value}')

In [ ]:
##find the gap between down/up time
print(max(train_logs["action_time"]), min(train_logs["action_time"]))

for i, r in train_logs.iterrows():
  cur_event = r["down_time"]